# *1. Cargamos los datos desde el Storage (PROCESSED) "Tabla DELTA" para realizar la ingenieria de caracteristicas*

In [0]:
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/processed/datmarcobre_prepro2_delta")
df_delta.show()

Convertimos el dataframe spark, a un DF de pandas, para realizar el trabajo

In [0]:
datos = df_delta.toPandas()
datos.head()

In [0]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
datos.dtypes

Filtramos solo las variables que usaremos en el proceso de ingenieria de caracteristicas

In [0]:
# 1. Filtramos
nuevos_nombres = {'radiohexagonocuchara_equipo_carguio' : 'radiohexagonocuchara_equipocarguio',
    'capacidad_vol_equipo_carguio':'capacidad_en_volumen_equipo_carguio_m3', 'capacidad_pes_equipo_carguio':'capacidad_en_peso_equipo_carguio'}

# Renombra las columnas del DataFrame
datos = datos.rename(columns=nuevos_nombres)

datos = datos[['tonelaje_inicial_poligono_creado',
'radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
'tiempo_ready_llegada_esperando',
'tiempo_ready_esperando_cuadra',
'tonelaje_camion_antes_cargaestabilizada',
'angulo_giro_promedio_pases', 'tonelaje_segun_computadora',
'id_equipo_camion', 'id_equipo_carguio',
'densidad_inicial_poligono_creado_tn/m3','numero_pases_carguio', 'productividad_operativa_acarreo_tn_h',
'tiempo_llegada_camion',
'tiempo_ready_cuadra_cuadrado',
'tiempo_ready_cuadrado_cargado',
'tiempo_ready_carga_acarreo',
'tiempo_ready_acarreo_cola',
 'tiempo_ready_cola_retro',
 'tiempo_ready_retro_listo',
 'tiempo_ready_listo_descarga',
 'tiempo_ready_descarga_viajandovacio',
 'tiempo_ready_cargando_pala',
 'tiempo_ready_esperando_pala',
'productividad_operativa_carguio_tn_h',
'al_termino_cargar_en_espera_cuadrado_cuadrandose',
'factor',
'capacidad_en_volumen_equipo_acarreo_m3']]

In [0]:
datos.shape

In [0]:
#- Redondeamos el valor del tonelaje inicial, a un solo decimal
datos['tonelaje_inicial_poligono_creado'] = datos['tonelaje_inicial_poligono_creado'].round(1)

#- Convertimos de float a int64 la variable id_equipo_carguio
datos['id_equipo_carguio'] = datos['id_equipo_carguio'].round(0).astype('int64')

In [0]:
datos.head()

In [0]:
%fs mounts

### Crear la BD dentro del Storage de Data Lake

In [0]:
# Crear la base de datos si no existe en el almacenamiento de PROCESSED
#spark.sql("CREATE DATABASE IF NOT EXISTS presentation_db LOCATION '/mnt/datalakemlopsd4m/presentation/'")
# n.1 Crear la base de datos si no existe en el almacenamiento de PRESENTATION
spark.sql("CREATE DATABASE IF NOT EXISTS proyectopases_presentation LOCATION '/mnt/datalakemlopsd4m/presentation/proyectopases_presentation/'")

In [0]:
spark.sql("SHOW DATABASES").show()

In [0]:
# n Antes de Guardar convertir el df-pandas Preprocesado a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos)

# n.1 Guardamos los datos preprocesados en el Storage de Processed , en una Tabla DELTA
spark_datos.write.format("delta").mode("overwrite").saveAsTable("presentation_db.datmarcobre_ingcaractv2_delta") #presentation:Nombre de BD y datos_processed_delta:Nombre